In [ ]:
!pip install cloudscraper
!pip install parsel

In [ ]:
!pip install cloudscraper --upgrade

In [ ]:
from parsel import Selector
import cloudscraper
import json
import pprint
import time
import requests
import csv
import random

In [ ]:
scraper = cloudscraper.create_scraper(
    browser={'browser': 'firefox', 'platform': 'windows', 'mobile': False}
)

In [ ]:
url = "https://www.tripadvisor.com.br/Restaurant_Review-g303518-d1065305-Reviews-or-Camaroes_Restaurante-Natal_State_of_Rio_Grande_do_Norte.html"
resp = scraper.get(url)
print(resp.status_code)

200


In [ ]:
def append_coments(elements):

    #filtrando os comentarios das respostas do dono da empresa
    filter = ["Gabriel Bezerra de Medeiros Camarões",
              "Gabriel B. Medeiros Camarões",
              "Medeiros Camarões",
              "Gabriel Bezerra",
              "Gabriel B.",
              "potiguar@camaroes.com.br",
              "thanks for the review",
              "Pedimos desculpas",
              "ouvidoria@camaroes.com.br",
              "Aguardamos seu retorno",
              "Esperamos revê-lo em breve por aqui"
              ]

    coments = []
    coments_removed = []
    cont_comnent = 0
    for element in elements:
        comentario = element.xpath('./text()').get()
        if comentario:
            coments_fit = comentario.strip().replace('\n', ' ')
            if not any(filtro.lower() in coments_fit.lower() for filtro in filter):
                coments.append(coments_fit.strip())
                cont_comnent+=1
            else:
                coments_removed.append(coments_fit.strip())
    if cont_comnent > 15:
        return 666

    return coments

In [ ]:
def append_titles(heads):
    titles = []
    for head in heads:
        cabeca = head.xpath('./text()').get()
        if cabeca:
            titles.append(cabeca.strip())
    return titles

In [ ]:
def count_stars(svgs):
    stars = []
    for svg in svgs:
        title_tag = svg.xpath('.//title')
        title_content = title_tag.xpath('./text()').get()
        stars.append(title_content)
    return stars

In [ ]:
i = 0
#i = 16380 #ultima pagina de comentarios
url = f"https://www.tripadvisor.com.br/Restaurant_Review-g303518-d1065305-Reviews-or{i}-Camaroes_Restaurante-Natal_State_of_Rio_Grande_do_Norte.html"

stars = []
coments = []
titles = []
pag = 1 #pagina


r = scraper.get(url)
response = Selector(text=r.text)

elements = response.xpath('//span[@class="JguWG"]') #comentarios
svgs = response.xpath('//div[@class="OSBmi J k"]//svg[@class="UctUV d H0"]') #estrelas
heads = response.xpath('//a[@class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS" and @rel="noopener"]') #titulos


inicio = time.time()

while elements:
    if r.status_code != 200:
        print("erro ao abrir a pagina")
        break
    if append_coments(elements) == 666:
        print(f"UM COMENTARIO A MAIS NA PAGINA {pag}")
        print(url)
        break
    else:
        coments.extend(append_coments(elements))
    titles.extend(append_titles(heads))
    stars.extend(count_stars(svgs))

    i+=15
    print(pag)
    url = f"https://www.tripadvisor.com.br/Restaurant_Review-g303518-d1065305-Reviews-or{i}-Camaroes_Restaurante-Natal_State_of_Rio_Grande_do_Norte.html"
    pag+=1
    if pag%30==0:
        time.sleep(random.randint(45,60))
    time.sleep(random.randint(5,20))
    r = scraper.get(url)
    response = Selector(text=r.text)

    elements = response.xpath('//span[@class="JguWG"]')
    svgs = response.xpath('//div[@class="OSBmi J k"]//svg[@class="UctUV d H0"]')
    heads = response.xpath('//a[@class="BMQDV _F Gv wSSLS SwZTJ FGwzt ukgoS" and @rel="noopener"]')

fim = time.time()
stars = [int(string[0]) for string in stars]

1
2
3
4


In [ ]:
tempo_total = fim - inicio
print(f"Tempo total de execução: {tempo_total} segundos")
print(f"Tempo total de execução: {tempo_total/60} minutos")

Tempo total de execução: 46.602620363235474 segundos
Tempo total de execução: 0.7767103393872579 minutos


In [ ]:
print(r.status_code)

200


In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
pp.pprint(coments)
print(len(coments))

[   'Restaurante muito perfeito!!! Preço justo, bem servido e comida '
    'maravilhosa.  Chegamos logo no horário que abriu e estava vazio, '
    'conseguimos escolher mesa e tudo, quando saímos deviam ter umas 30 '
    'pessoas aguardando para entrar. O camarão internacional comem 3 pessoas',
    'O restaurante é excelente, ambiente lindo, limpo e agradável. Os pratos '
    'são muito saborosos, bem servidos e com custo benefício excelente. Na '
    'primeira visita comemos o Ceviche de peixe e camarão delicioso e a '
    'Lagosta Ponta Negra que estava perfeita. No segundo dia comemos novamente '
    'o Ceviche (vale a pena) e o Camarão 294, entradinha super bem servida.',
    'Excelente lugar para novas experiências com camarões e pratos de carne. '
    'Atendimento muito rápido e atencioso! Voltarei mais vezes.',
    'Esse restaurante é o melhor de todos que já fui, custo beneficio ok, e '
    'comida surreais de gostosas. A sobremesa naked de ninho ganhou meu '
    'coração, já f

In [ ]:
pp.pprint(titles)
print(len(titles))

[   'Perfeito',
    'Sabor impecável',
    'Ótimo restaurante para ter experiências.',
    'Restaurante perfeito!',
    'familia em férias',
    'O MELHOR RESTAURANTE DE FRUTOS DO MAR',
    'Comida deliciosa..caipirinha não recomendo',
    'Camarões',
    'Comida excelente',
    'Camarão é muito bom',
    'Comida maravilhosa',
    'UM RESTAURANTE EXCEPCIONAL!',
    'Merece por conta da tradição',
    'Melhor escolha em Ponta Negra',
    'Comida muito boa',
    'Perfeição existe!',
    'Almoço com família e amigos',
    'Vale muito',
    'Experiência espetacular.',
    'Parada obrigatória!',
    'Restaurante que faz juz à fama que quem!',
    'Excelente',
    'O melhor camarão da vida!',
    'Tem que colocar no seu roteiro',
    'Excelente pedida em Natal',
    'Indico sem medo',
    'Ótimo restaurante com preço justo',
    'O mais bem afamado da região',
    'Um clássico obrigatório para ir em Natal',
    'Recomendo',
    'Melhor de Ponta Negra !!!',
    'Pratos maravilhosos',
    'A e

In [ ]:
print(stars)
print(len(stars))

[5, 5, 5, 5, 4, 5, 4, 5, 5, 5, 5, 5, 4, 5, 5, 5, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 5, 1, 5, 5, 5, 5, 5, 5, 2, 5, 5, 4, 5, 5, 5, 5, 4, 5, 1, 5, 4, 5, 5]
60


In [ ]:
matriz_csv = [['Estrelas', 'Títulos', 'Comentários']]
matriz_csv.extend([[estrela, titulo, comentario] for estrela, titulo, comentario in zip(stars, titles, coments)])

pp.pprint(matriz_csv)

[   ['Estrelas', 'Títulos', 'Comentários'],
    [   5,
        'Perfeito',
        'Restaurante muito perfeito!!! Preço justo, bem servido e comida '
        'maravilhosa.  Chegamos logo no horário que abriu e estava vazio, '
        'conseguimos escolher mesa e tudo, quando saímos deviam ter umas 30 '
        'pessoas aguardando para entrar. O camarão internacional comem 3 '
        'pessoas'],
    [   5,
        'Sabor impecável',
        'O restaurante é excelente, ambiente lindo, limpo e agradável. Os '
        'pratos são muito saborosos, bem servidos e com custo benefício '
        'excelente. Na primeira visita comemos o Ceviche de peixe e camarão '
        'delicioso e a Lagosta Ponta Negra que estava perfeita. No segundo dia '
        'comemos novamente o Ceviche (vale a pena) e o Camarão 294, entradinha '
        'super bem servida.'],
    [   5,
        'Ótimo restaurante para ter experiências.',
        'Excelente lugar para novas experiências com camarões e pratos de '
   

In [ ]:
with open('/content/drive/MyDrive/Machine Learning Project/csv_folder/camaroes_.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Estrelas', 'Título', 'Comentarios'])

    for i in range(len(coments)):
        writer.writerow([i+1,stars[i], titles[i], coments[i]])